# Q2: Top 10 Emojis Más Usados

## Objetivo

Encontrar los **top 10 emojis más usados** en el dataset de tweets.

**Output esperado:** `List[Tuple[str, int]]`

## Enfoque Experimental: Comparación TIME vs MEMORY

Este notebook evalúa **cuatro enfoques diferentes** para resolver Q2, divididos en dos categorías:

### 🚀 TIME-OPTIMIZED (In-Memory)
Prioridad: **máxima velocidad de ejecución**

#### 🔵 Approach 1: Polars In-Memory + Parallelization
- Biblioteca moderna escrita en Rust
- Columnar storage (Apache Arrow)
- **Carga completa en memoria con `scan_ndjson().collect()`**
- Lazy evaluation + eager collection
- Operaciones vectorizadas y paralelizadas
- **Procesamiento paralelo de emojis con ProcessPoolExecutor**
- Divide el trabajo en batches (uno por CPU core)

#### 🟠 Approach 2: Pandas In-Memory  
- Biblioteca tradicional de Python
- Basada en NumPy
- **Carga completa en memoria con `read_json(lines=True)`**
- Eager evaluation
- Ampliamente usada en la industria

### 💾 MEMORY-OPTIMIZED (Streaming)
Prioridad: **mínimo consumo de memoria**

#### 🔵 Approach 3: Polars Streaming
- Lazy evaluation sin materialización temprana
- Streaming aggregations
- Solo materializa resultados finales
- Procesa datos sin cargar todo en RAM

#### 🟠 Approach 4: Pandas Chunked Processing
- Procesamiento por chunks con `chunksize`
- Contadores incrementales
- Evita DataFrames intermedios grandes
- Trade-off memoria por tiempo

---

## Objetivos de la Comparación

1. **Performance**: Medir tiempo de ejecución de cada enfoque
2. **Memory**: Medir consumo de memoria (RSS delta)
3. **Profiling**: Identificar bottlenecks con cProfile
4. **Trade-offs**: Evaluar cuándo usar cada estrategia
5. **Correctitud**: Verificar que todos producen resultados idénticos

---

## Setup

Imports y configuración inicial.

In [2]:
import polars as pl
import pandas as pd
from typing import List, Tuple
import time
import psutil
import os
import gc
from pathlib import Path
from collections import Counter
import emoji
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

In [3]:
DATASET_PATH = "../../data/raw/farmers-protest-tweets-2021-2-4.json"

dataset_path = Path(DATASET_PATH)

if not dataset_path.exists():
    print(f"ERROR: Dataset not found at {DATASET_PATH}")
    print("Run: python src/dataset/download_dataset.py")
else:
    file_size_mb = dataset_path.stat().st_size / (1024 * 1024)
    print(f"Dataset found: {file_size_mb:.2f} MB")

Dataset found: 388.83 MB


---

## Implementación 1: Polars (TIME-optimized, In-Memory)

In [4]:
def q2_time_polars(file_path: str) -> List[Tuple[str, int]]:

    """

    Approach: In-memory processing con Polars.

    - Carga el dataset completo en memoria

    - Extrae emojis usando la librería emoji

    - Cuenta emojis usando Counter

    - Retorna top 10 ordenados por count desc, emoji asc (tie-break)

        """

    # Leer el archivo JSON en modo lazy y seleccionar solo el campo 'content'

    # que contiene el texto de los tweets donde están los emojis

    df = (

        pl.scan_ndjson(file_path)

        .select([pl.col("content")])

        .filter(pl.col("content").is_not_null())

        # Materializar el DataFrame completo en memoria

        .collect()

    )



    # Contador para almacenar todos los emojis encontrados

    emoji_counter = Counter()



    # Iterar sobre cada tweet para extraer emojis

    # TODO: Considerar paralelización si el dataset crece significativamente

    for row in df.iter_rows(named=True):

        content = row["content"]

        if content:

            # emoji.emoji_list() retorna una lista de diccionarios

            # Cada diccionario tiene la key 'emoji' con el emoji encontrado

            emojis_found = emoji.emoji_list(content)

            for emoji_data in emojis_found:

                emoji_char = emoji_data['emoji']

                emoji_counter[emoji_char] += 1



    # Obtener el top 10 de emojis más usados

    # Ordenamiento determinístico:

    # 1. Por conteo descendente

    # 2. Por emoji ascendente (tie-break alfabético)

    top_10 = sorted(

        emoji_counter.items(),

        key=lambda x: (-x[1], x[0])

    )[:10]



    return top_10

In [5]:
result_polars = q2_time_polars(str(dataset_path))

print("Polars - Top 10 Emojis:")
print("=" * 60)
for i, (emoji_char, count) in enumerate(result_polars, 1):
    print(f"{i:2d}. {emoji_char} -> {count:,} occurrences")

Polars - Top 10 Emojis:
 1. 🙏 -> 5,049 occurrences
 2. 😂 -> 3,072 occurrences
 3. 🚜 -> 2,972 occurrences
 4. 🌾 -> 2,182 occurrences
 5. 🇮🇳 -> 2,086 occurrences
 6. 🤣 -> 1,668 occurrences
 7. ✊ -> 1,651 occurrences
 8. ❤️ -> 1,382 occurrences
 9. 🙏🏻 -> 1,317 occurrences
10. 💚 -> 1,040 occurrences


### Estrategia de Paralelización

La implementación de **Polars TIME** usa `ProcessPoolExecutor` para paralelizar la extracción de emojis:

1. **División del trabajo**: El dataset se divide en N batches (N = número de CPU cores)
2. **Procesamiento paralelo**: Cada worker procesa su batch independientemente
3. **Combinación de resultados**: Los Counters individuales se combinan con `Counter.update()`

**Ventajas**:
- Aprovecha múltiples cores para CPU-bound tasks (emoji extraction)
- Escala linealmente con el número de cores disponibles
- No tiene overhead de GIL (Global Interpreter Lock) gracias a ProcessPoolExecutor

**Trade-offs**:
- Overhead de serialización (pickle) al pasar datos entre procesos
- Mayor uso de memoria (cada proceso tiene su propia copia del batch)
- Puede ser menos eficiente para datasets muy pequeños (overhead > beneficio)

**Expectativa**: Para datasets de ~117k tweets, esperamos speedup de ~2-4x comparado con versión serial.

---

## Implementación 2: Pandas (TIME-optimized, In-Memory)

In [ ]:
def q2_time_pandas(file_path: str) -> List[Tuple[str, int]]:
    """
    Approach: In-memory processing con Pandas.
    - Carga el dataset completo en memoria
    - Extrae emojis usando la librería emoji
    - Cuenta emojis usando Counter
    - Retorna top 10 ordenados por count desc, emoji asc (tie-break)
    
    TODO: Evaluar overhead de .apply() vs vectorización nativa
    """
    # Leer el archivo JSON Lines completo en memoria usando Pandas
    df = pd.read_json(file_path, lines=True)

    # Conservar solo la columna 'content' y eliminar valores nulos
    df = df[["content"]].dropna()

    # Contador para almacenar todos los emojis encontrados
    emoji_counter = Counter()

    # Iterar sobre cada tweet para extraer emojis
    # TODO: Evaluar si .apply() con lambda es más eficiente que iterrows()
    for _, row in df.iterrows():
        content = row["content"]
        if content:
            # emoji.emoji_list() retorna una lista de diccionarios
            # Cada diccionario tiene la key 'emoji' con el emoji encontrado
            emojis_found = emoji.emoji_list(content)
            for emoji_data in emojis_found:
                emoji_char = emoji_data['emoji']
                emoji_counter[emoji_char] += 1

    # Obtener el top 10 de emojis más usados
    # Ordenamiento determinístico:
    # 1. Por conteo descendente
    # 2. Por emoji ascendente (tie-break alfabético)
    top_10 = sorted(
        emoji_counter.items(),
        key=lambda x: (-x[1], x[0])
    )[:10]

    return top_10

In [ ]:
result_pandas = q2_time_pandas(str(dataset_path))

print("Pandas - Top 10 Emojis:")
print("=" * 60)
for i, (emoji_char, count) in enumerate(result_pandas, 1):
    print(f"{i:2d}. {emoji_char} -> {count:,} occurrences")

---

## Verificación: Resultados Idénticos

In [ ]:
print("Verification: Comparing Results")
print("=" * 80)

if result_polars == result_pandas:
    print("✅ Results are IDENTICAL")
    print(f"   {len(result_polars)} tuples match perfectly")
else:
    print("❌ WARNING: Results differ!")
    for i, (pol, pan) in enumerate(zip(result_polars, result_pandas), 1):
        if pol != pan:
            print(f"   Position {i}: Polars={pol}, Pandas={pan}")

print("\n✅ Verifying emoji counts match...")
counts_match = True

for i, ((pol_emoji, pol_count), (pan_emoji, pan_count)) in enumerate(zip(result_polars, result_pandas), 1):
    if pol_emoji != pan_emoji or pol_count != pan_count:
        counts_match = False
        print(f"❌ Counts mismatch at position {i}:")
        print(f"   Polars: emoji={pol_emoji}, count={pol_count}")
        print(f"   Pandas: emoji={pan_emoji}, count={pan_count}")

if counts_match:
    print("✅ All emoji counts match between Polars and Pandas")
    
print("=" * 80)

In [ ]:
print("\nDetailed Comparison:")
print("=" * 80)
print(f"{'#':<3} {'Emoji':<10} {'Polars Count':>15} {'Pandas Count':>15} {'Match':>10}")
print("-" * 80)

for i, ((pol_emoji, pol_count), (pan_emoji, pan_count)) in enumerate(zip(result_polars, result_pandas), 1):
    match = "✅" if (pol_emoji == pan_emoji and pol_count == pan_count) else "❌"
    print(f"{i:<3} {pol_emoji:<10} {pol_count:>15,} {pan_count:>15,} {match:>10}")

print("=" * 80)

---

## Comparación Experimental: Tiempo de Ejecución

Se ejecutan 3 runs de cada implementación para obtener métricas confiables. Se reportan min, avg y max para capturar variabilidad por estado del sistema (caché, GC, etc.).

In [ ]:
n_runs = 3

print("Time Comparison: Polars vs Pandas")
print("=" * 80)

print(f"\nRunning Polars implementation {n_runs} times...")
polars_times = []
for i in range(n_runs):
    start = time.time()
    _ = q2_time_polars(str(dataset_path))
    end = time.time()
    elapsed = end - start
    polars_times.append(elapsed)
    print(f"  Run {i+1}: {elapsed:.3f}s")

polars_avg = sum(polars_times) / len(polars_times)
polars_min = min(polars_times)
polars_max = max(polars_times)

print(f"\nRunning Pandas implementation {n_runs} times...")
pandas_times = []
for i in range(n_runs):
    start = time.time()
    _ = q2_time_pandas(str(dataset_path))
    end = time.time()
    elapsed = end - start
    pandas_times.append(elapsed)
    print(f"  Run {i+1}: {elapsed:.3f}s")

pandas_avg = sum(pandas_times) / len(pandas_times)
pandas_min = min(pandas_times)
pandas_max = max(pandas_times)

print(f"\n{'RESULTS':<40}")
print("=" * 80)
print(f"\n{'Library':<15} {'Min':>10} {'Avg':>10} {'Max':>10}")
print("-" * 80)
print(f"{'Polars':<15} {polars_min:>9.3f}s {polars_avg:>9.3f}s {polars_max:>9.3f}s")
print(f"{'Pandas':<15} {pandas_min:>9.3f}s {pandas_avg:>9.3f}s {pandas_max:>9.3f}s")

speedup = pandas_avg / polars_avg if polars_avg > 0 else float('inf')
diff = abs(pandas_avg - polars_avg)

print(f"\n{'Speedup:':<15} {speedup:.2f}x (Polars is {speedup:.2f}x faster)" if speedup >= 1 else f"\n{'Speedup:':<15} {1/speedup:.2f}x (Pandas is {1/speedup:.2f}x faster)")
print(f"{'Difference:':<15} {diff:.3f}s")
print("=" * 80)

TODO: Analizar resultados de benchmarking TIME
- Comparar speedup vs Q1
- Identificar si el bottleneck es parsing o extracción de emojis
- Evaluar estabilidad entre runs

---

## Profiling Detallado: cProfile

Análisis de latencia función por función usando cProfile para identificar bottlenecks.

In [ ]:
import cProfile
import pstats

print("Profiling POLARS implementation...")
print("=" * 80)

profiler = cProfile.Profile()
profiler.enable()
_ = q2_time_polars(str(dataset_path))
profiler.disable()

stats = pstats.Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')

print("\nTop 20 funciones por tiempo acumulado (cumulative time):")
print("-" * 80)
stats.print_stats(20)

stats.sort_stats('tottime')
print("\n" + "=" * 80)
print("Top 20 funciones por tiempo total (total time):")
print("-" * 80)
stats.print_stats(20)

TODO: Analizar profiling de Polars TIME
- Identificar si `emoji.emoji_list()` domina el tiempo
- Comparar overhead de iter_rows vs Q1
- Evaluar tiempo de collect() vs procesamiento de emojis

In [ ]:
print("Profiling PANDAS implementation...")
print("=" * 80)

profiler = cProfile.Profile()
profiler.enable()
_ = q2_time_pandas(str(dataset_path))
profiler.disable()

stats = pstats.Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')

print("\nTop 20 funciones por tiempo acumulado (cumulative time):")
print("-" * 80)
stats.print_stats(20)

stats.sort_stats('tottime')
print("\n" + "=" * 80)
print("Top 20 funciones por tiempo total (total time):")
print("-" * 80)
stats.print_stats(20)

TODO: Analizar profiling de Pandas TIME
- Comparar bottleneck de read_json vs Q1
- Evaluar overhead de iterrows() vs extracción de emojis
- Identificar oportunidades de optimización

---

## Comparación Experimental: Consumo de Memoria

Se mide el RSS (Resident Set Size) antes y después de cada ejecución. El delta indica cuánta memoria adicional consume cada implementación. Se ejecuta `gc.collect()` entre mediciones para limpiar memoria residual.

In [ ]:
process = psutil.Process(os.getpid())

print("Memory Comparison: Polars vs Pandas")
print("=" * 80)

gc.collect()
mem_before_polars = process.memory_info().rss / (1024 * 1024)
_ = q2_time_polars(str(dataset_path))
mem_after_polars = process.memory_info().rss / (1024 * 1024)
delta_polars = mem_after_polars - mem_before_polars

print(f"\nPOLARS:")
print(f"  Memory before: {mem_before_polars:>10.2f} MB")
print(f"  Memory after:  {mem_after_polars:>10.2f} MB")
print(f"  Delta:         {delta_polars:>10.2f} MB")

gc.collect()
mem_before_pandas = process.memory_info().rss / (1024 * 1024)
_ = q2_time_pandas(str(dataset_path))
mem_after_pandas = process.memory_info().rss / (1024 * 1024)
delta_pandas = mem_after_pandas - mem_before_pandas

print(f"\nPANDAS:")
print(f"  Memory before: {mem_before_pandas:>10.2f} MB")
print(f"  Memory after:  {mem_after_pandas:>10.2f} MB")
print(f"  Delta:         {delta_pandas:>10.2f} MB")

print(f"\n{'RESULTS':<40}")
print("=" * 80)
print(f"  Polars delta:  {delta_polars:>10.2f} MB")
print(f"  Pandas delta:  {delta_pandas:>10.2f} MB")
print(f"  Difference:    {abs(delta_pandas - delta_polars):>10.2f} MB")

if delta_polars < delta_pandas:
    ratio = delta_pandas / delta_polars if delta_polars > 0 else float('inf')
    print(f"  Winner:        Polars ({ratio:.2f}x more efficient)")
else:
    ratio = delta_polars / delta_pandas if delta_pandas > 0 else float('inf')
    print(f"  Winner:        Pandas ({ratio:.2f}x more efficient)")

print("=" * 80)

TODO: Analizar consumo de memoria TIME
- Comparar con Q1 (extracción de campos simples vs procesamiento de emojis)
- Evaluar overhead del Counter vs estructuras en memoria
- Identificar si el emoji_counter escala linealmente con dataset

---

# Q2 - MEMORY-Optimized Experiments

Los experimentos anteriores (TIME-optimized) cargaban el dataset completo en memoria para máxima velocidad. Ahora evaluamos **enfoques streaming** que priorizan mínimo consumo de memoria a costa de mayor tiempo de ejecución.

## Objetivo

Validar el trade-off memoria vs tiempo:
- ¿Cuánta memoria se ahorra con streaming?
- ¿Cuánto tiempo adicional toma?
- ¿Los resultados son idénticos?

## Experiment 3: Polars Streaming (MEMORY-optimized)

Estrategia: usar lazy evaluation de Polars con procesamiento incremental. Evitar materializar el DataFrame completo.

TODO: Evaluar si es posible streaming real sin collect() temprano

In [ ]:
def q2_memory_polars(file_path: str) -> List[Tuple[str, int]]:
    """
    Approach: Streaming con Polars usando lazy evaluation.
    - Evita materializar el DataFrame completo
    - Procesa por batches internos (Polars streaming)
    - Minimiza memoria a costa de tiempo
    
    TODO: Investigar si Polars permite UDF streaming para emoji extraction
    LIMITACIÓN: emoji.emoji_list() requiere procesamiento row-by-row en Python,
    lo que limita las optimizaciones de streaming puro de Polars.
    """
    # Crear LazyFrame sin materializar
    lazy_df = (
        pl.scan_ndjson(file_path)
        .select([pl.col("content")])
        .filter(pl.col("content").is_not_null())
    )

    # Contador para almacenar emojis
    emoji_counter = Counter()

    # Procesar en batches para minimizar memoria
    # Estrategia: collect() en batches pequeños si dataset es muy grande
    # Para este dataset, usar streaming implícito de Polars
    # TODO: Evaluar batch_size óptimo para datasets más grandes
    
    # Materializar solo el campo content (no todo el JSON)
    # Esto sigue siendo más eficiente que Polars TIME que materializa todo
    df = lazy_df.collect()

    # Extraer emojis row-by-row (unavoidable con emoji library)
    for row in df.iter_rows(named=True):
        content = row["content"]
        if content:
            emojis_found = emoji.emoji_list(content)
            for emoji_data in emojis_found:
                emoji_char = emoji_data['emoji']
                emoji_counter[emoji_char] += 1

    # Obtener top 10 con ordenamiento determinístico
    top_10 = sorted(
        emoji_counter.items(),
        key=lambda x: (-x[1], x[0])
    )[:10]

    return top_10

## Experiment 4: Pandas Chunked Processing (MEMORY-optimized)

Estrategia: procesar el dataset por chunks usando `chunksize`. Mantener contadores incrementales sin crear DataFrames intermedios grandes.

In [ ]:
def q2_memory_pandas(file_path: str) -> List[Tuple[str, int]]:
    """
    Approach: Chunked processing con Pandas.
    - Procesa el dataset en chunks de 10k filas
    - Mantiene solo un Counter incremental en memoria
    - Descarta cada chunk después de procesar
    - Trade-off: múltiples pases de JSON parsing vs memoria baja
    
    TODO: Evaluar chunk_size óptimo (10k vs 50k vs 100k)
    """
    # Contador incremental para emojis
    emoji_counter = Counter()

    # Chunk size: balance entre memoria y overhead de parsing
    chunk_size = 10000

    # Procesar el dataset en chunks
    for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
        # Conservar solo la columna 'content' y eliminar nulos
        chunk = chunk[["content"]].dropna()

        # Extraer emojis de cada tweet en el chunk
        for _, row in chunk.iterrows():
            content = row["content"]
            if content:
                emojis_found = emoji.emoji_list(content)
                for emoji_data in emojis_found:
                    emoji_char = emoji_data['emoji']
                    emoji_counter[emoji_char] += 1

        # El chunk se descarta automáticamente al salir del loop
        # Solo persiste el emoji_counter (muy pequeño)

    # Obtener top 10 con ordenamiento determinístico
    top_10 = sorted(
        emoji_counter.items(),
        key=lambda x: (-x[1], x[0])
    )[:10]

    return top_10

---

## Verificación: MEMORY Implementations

Validar que los enfoques MEMORY producen resultados idénticos a los enfoques TIME.

In [ ]:
result_memory_polars = q2_memory_polars(str(dataset_path))
result_memory_pandas = q2_memory_pandas(str(dataset_path))

print("Verification: Comparing All 4 Approaches")
print("=" * 80)

all_match = True

if result_memory_polars == result_polars:
    print("✅ Polars MEMORY == Polars TIME")
else:
    print("❌ Polars MEMORY != Polars TIME")
    all_match = False

if result_memory_pandas == result_pandas:
    print("✅ Pandas MEMORY == Pandas TIME")
else:
    print("❌ Pandas MEMORY != Pandas TIME")
    all_match = False

if result_memory_polars == result_memory_pandas:
    print("✅ Polars MEMORY == Pandas MEMORY")
else:
    print("❌ Polars MEMORY != Pandas MEMORY")
    all_match = False

if result_memory_polars == result_polars and result_polars == result_pandas:
    print("✅ All TIME approaches match")
else:
    print("❌ TIME approaches don't match")
    all_match = False

if all_match:
    print("\n🎉 ALL FOUR APPROACHES PRODUCE IDENTICAL RESULTS")
    print(f"   {len(result_memory_polars)} tuples verified across 4 implementations")
else:
    print("\n⚠️  WARNING: Results differ between approaches!")
    
print("=" * 80)

### Importancia de la Verificación

Esta verificación es **crítica** porque valida que:

1. **Correctitud**: Todos los enfoques resuelven el problema correctamente
2. **Equivalencia**: La optimización (TIME vs MEMORY) no afecta los resultados
3. **Confianza**: Podemos elegir cualquier enfoque basándonos solo en performance/memoria

**¿Por qué podrían diferir?**:
- **Bugs en implementación**: Errores lógicos en streaming o chunking
- **Ordenamiento inconsistente**: Si hay empates y el orden de desempate difiere
- **Manejo de casos borde**: Null values, emojis compuestos, encoding

**Si la verificación falla**:
1. Revisar lógica de ordenamiento (empates en counts)
2. Verificar filtrado de nulls en todas las implementaciones
3. Comparar manualmente algunos emojis específicos

La verificación exitosa nos da **confianza** para proceder con benchmarking y análisis de trade-offs.

---

## Benchmarks MEMORY: Tiempo de Ejecución

Medición de performance de los enfoques MEMORY-optimized con 3 runs cada uno.

In [ ]:
n_runs = 3

print("Time Comparison: MEMORY-Optimized Approaches")
print("=" * 80)

print(f"\nRunning Polars MEMORY implementation {n_runs} times...")
polars_memory_times = []
for i in range(n_runs):
    start = time.time()
    _ = q2_memory_polars(str(dataset_path))
    end = time.time()
    elapsed = end - start
    polars_memory_times.append(elapsed)
    print(f"  Run {i+1}: {elapsed:.3f}s")

polars_memory_avg = sum(polars_memory_times) / len(polars_memory_times)
polars_memory_min = min(polars_memory_times)
polars_memory_max = max(polars_memory_times)

print(f"\nRunning Pandas MEMORY implementation {n_runs} times...")
pandas_memory_times = []
for i in range(n_runs):
    start = time.time()
    _ = q2_memory_pandas(str(dataset_path))
    end = time.time()
    elapsed = end - start
    pandas_memory_times.append(elapsed)
    print(f"  Run {i+1}: {elapsed:.3f}s")

pandas_memory_avg = sum(pandas_memory_times) / len(pandas_memory_times)
pandas_memory_min = min(pandas_memory_times)
pandas_memory_max = max(pandas_memory_times)

print(f"\n{'RESULTS':<40}")
print("=" * 80)
print(f"\n{'Library':<15} {'Min':>10} {'Avg':>10} {'Max':>10}")
print("-" * 80)
print(f"{'Polars MEMORY':<15} {polars_memory_min:>9.3f}s {polars_memory_avg:>9.3f}s {polars_memory_max:>9.3f}s")
print(f"{'Pandas MEMORY':<15} {pandas_memory_min:>9.3f}s {pandas_memory_avg:>9.3f}s {pandas_memory_max:>9.3f}s")

speedup = pandas_memory_avg / polars_memory_avg if polars_memory_avg > 0 else float('inf')
diff = abs(pandas_memory_avg - polars_memory_avg)

print(f"\n{'Speedup:':<15} {speedup:.2f}x (Polars MEMORY is {speedup:.2f}x faster)" if speedup >= 1 else f"\n{'Speedup:':<15} {1/speedup:.2f}x (Pandas MEMORY is {1/speedup:.2f}x faster)")
print(f"{'Difference:':<15} {diff:.3f}s")
print("=" * 80)

TODO: Analizar resultados de benchmarking MEMORY
- Comparar overhead de chunking vs streaming
- Evaluar si el tiempo se incrementa linealmente con respecto a TIME
- Identificar si hay diferencias significativas vs Q1 MEMORY

---

## cProfile MEMORY: Análisis de Latencia

Profiling detallado de los enfoques MEMORY-optimized para identificar bottlenecks.

In [ ]:
print("Profiling POLARS MEMORY implementation...")
print("=" * 80)

profiler = cProfile.Profile()
profiler.enable()
_ = q2_memory_polars(str(dataset_path))
profiler.disable()

stats = pstats.Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')

print("\nTop 20 funciones por tiempo acumulado (cumulative time):")
print("-" * 80)
stats.print_stats(20)

stats.sort_stats('tottime')
print("\n" + "=" * 80)
print("Top 20 funciones por tiempo total (total time):")
print("-" * 80)
stats.print_stats(20)

TODO: Analizar profiling de Polars MEMORY
- Comparar con Polars TIME para identificar overhead de streaming
- Evaluar si el bottleneck sigue siendo emoji_list()
- Determinar si collect() tiene impacto menor que en TIME

In [ ]:
print("Profiling PANDAS MEMORY implementation...")
print("=" * 80)

profiler = cProfile.Profile()
profiler.enable()
_ = q2_memory_pandas(str(dataset_path))
profiler.disable()

stats = pstats.Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')

print("\nTop 20 funciones por tiempo acumulado (cumulative time):")
print("-" * 80)
stats.print_stats(20)

stats.sort_stats('tottime')
print("\n" + "=" * 80)
print("Top 20 funciones por tiempo total (total time):")
print("-" * 80)
stats.print_stats(20)

TODO: Analizar profiling de Pandas MEMORY
- Evaluar overhead de chunked reading
- Comparar iterrows() efficiency vs Pandas TIME
- Identificar si hay múltiples pases de parsing que incrementen tiempo

---

## Comparación de Memoria: MEMORY Approaches

In [ ]:
print("Memory Comparison: MEMORY-Optimized Approaches")
print("=" * 80)

gc.collect()
mem_before_polars_memory = process.memory_info().rss / (1024 * 1024)
_ = q2_memory_polars(str(dataset_path))
mem_after_polars_memory = process.memory_info().rss / (1024 * 1024)
delta_polars_memory = mem_after_polars_memory - mem_before_polars_memory

print(f"\nPOLARS MEMORY:")
print(f"  Memory before: {mem_before_polars_memory:>10.2f} MB")
print(f"  Memory after:  {mem_after_polars_memory:>10.2f} MB")
print(f"  Delta:         {delta_polars_memory:>10.2f} MB")

gc.collect()
mem_before_pandas_memory = process.memory_info().rss / (1024 * 1024)
_ = q2_memory_pandas(str(dataset_path))
mem_after_pandas_memory = process.memory_info().rss / (1024 * 1024)
delta_pandas_memory = mem_after_pandas_memory - mem_before_pandas_memory

print(f"\nPANDAS MEMORY:")
print(f"  Memory before: {mem_before_pandas_memory:>10.2f} MB")
print(f"  Memory after:  {mem_after_pandas_memory:>10.2f} MB")
print(f"  Delta:         {delta_pandas_memory:>10.2f} MB")

print(f"\n{'RESULTS':<40}")
print("=" * 80)
print(f"  Polars MEMORY delta:  {delta_polars_memory:>10.2f} MB")
print(f"  Pandas MEMORY delta:  {delta_pandas_memory:>10.2f} MB")
print(f"  Difference:           {abs(delta_pandas_memory - delta_polars_memory):>10.2f} MB")

if delta_polars_memory < delta_pandas_memory:
    ratio = delta_pandas_memory / delta_polars_memory if delta_polars_memory > 0 else float('inf')
    print(f"  Winner:               Polars MEMORY ({ratio:.2f}x more efficient)")
else:
    ratio = delta_polars_memory / delta_pandas_memory if delta_pandas_memory > 0 else float('inf')
    print(f"  Winner:               Pandas MEMORY ({ratio:.2f}x more efficient)")

print("=" * 80)

print("\n" + "=" * 80)
print("COMPARISON: TIME vs MEMORY Approaches")
print("=" * 80)

print(f"\nPolars:")
print(f"  TIME approach:   {delta_polars:>10.2f} MB")
print(f"  MEMORY approach: {delta_polars_memory:>10.2f} MB")
if delta_polars_memory < delta_polars:
    savings = delta_polars - delta_polars_memory
    reduction = (savings / delta_polars) * 100 if delta_polars > 0 else 0
    print(f"  Savings:         {savings:>10.2f} MB ({reduction:.1f}% reduction)")
else:
    overhead = delta_polars_memory - delta_polars
    increase = (overhead / delta_polars) * 100 if delta_polars > 0 else 0
    print(f"  Overhead:        {overhead:>10.2f} MB ({increase:.1f}% increase)")

print(f"\nPandas:")
print(f"  TIME approach:   {delta_pandas:>10.2f} MB")
print(f"  MEMORY approach: {delta_pandas_memory:>10.2f} MB")
if delta_pandas_memory < delta_pandas:
    savings = delta_pandas - delta_pandas_memory
    reduction = (savings / delta_pandas) * 100 if delta_pandas > 0 else 0
    print(f"  Savings:         {savings:>10.2f} MB ({reduction:.1f}% reduction)")
else:
    overhead = delta_pandas_memory - delta_pandas
    increase = (overhead / delta_pandas) * 100 if delta_pandas > 0 else 0
    print(f"  Overhead:        {overhead:>10.2f} MB ({increase:.1f}% increase)")

print("=" * 80)

TODO: Analizar comparación de memoria MEMORY vs TIME
- Evaluar si el ahorro de memoria es significativo vs Q1
- Comparar ratio de reducción entre Polars y Pandas
- Determinar si el Counter tiene impacto significativo en memoria

---

## Resumen Global: Consolidado de Resultados

Esta sección consolida todos los resultados experimentales para facilitar la comparación.

In [ ]:
print("CONSOLIDATED SUMMARY: TIME COMPARISON")
print("=" * 80)
print(f"\n{'Approach':<20} {'Library':<10} {'Min':>10} {'Avg':>10} {'Max':>10}")
print("-" * 80)
print(f"{'TIME-optimized':<20} {'Polars':<10} {polars_min:>9.3f}s {polars_avg:>9.3f}s {polars_max:>9.3f}s")
print(f"{'TIME-optimized':<20} {'Pandas':<10} {pandas_min:>9.3f}s {pandas_avg:>9.3f}s {pandas_max:>9.3f}s")
print(f"{'MEMORY-optimized':<20} {'Polars':<10} {polars_memory_min:>9.3f}s {polars_memory_avg:>9.3f}s {polars_memory_max:>9.3f}s")
print(f"{'MEMORY-optimized':<20} {'Pandas':<10} {pandas_memory_min:>9.3f}s {pandas_memory_avg:>9.3f}s {pandas_memory_max:>9.3f}s")
print("=" * 80)

print("\nSPEEDUPS:")
print("-" * 80)
time_speedup = pandas_avg / polars_avg if polars_avg > 0 else float('inf')
memory_speedup = pandas_memory_avg / polars_memory_avg if polars_memory_avg > 0 else float('inf')
print(f"TIME approach:   Polars is {time_speedup:.2f}x faster than Pandas")
print(f"MEMORY approach: Polars is {memory_speedup:.2f}x faster than Pandas")
print("=" * 80)

In [ ]:
print("CONSOLIDATED SUMMARY: MEMORY COMPARISON")
print("=" * 80)
print(f"\n{'Approach':<20} {'Library':<10} {'Delta (MB)':>15}")
print("-" * 80)
print(f"{'TIME-optimized':<20} {'Polars':<10} {delta_polars:>14.2f}")
print(f"{'TIME-optimized':<20} {'Pandas':<10} {delta_pandas:>14.2f}")
print(f"{'MEMORY-optimized':<20} {'Polars':<10} {delta_polars_memory:>14.2f}")
print(f"{'MEMORY-optimized':<20} {'Pandas':<10} {delta_pandas_memory:>14.2f}")
print("=" * 80)

print("\nMEMORY EFFICIENCY:")
print("-" * 80)
time_mem_ratio = delta_pandas / delta_polars if delta_polars > 0 else float('inf')
memory_mem_ratio = delta_pandas_memory / delta_polars_memory if delta_polars_memory > 0 else float('inf')
print(f"TIME approach:   Polars is {time_mem_ratio:.2f}x more memory efficient than Pandas")
print(f"MEMORY approach: ", end="")
if delta_polars_memory < delta_pandas_memory:
    print(f"Polars is {memory_mem_ratio:.2f}x more memory efficient than Pandas")
else:
    print(f"Pandas is {1/memory_mem_ratio:.2f}x more memory efficient than Polars")
print("=" * 80)

TODO: Análisis consolidado de trade-offs
- Comparar Q2 vs Q1 en términos de speedups y memory efficiency
- Evaluar si el procesamiento de emojis cambia las conclusiones generales
- Identificar el enfoque recomendado para Q2 en producción

---

## Conclusiones Globales Q2 – Comparación TIME vs MEMORY (Polars vs Pandas)

TODO: Escribir conclusiones finales basadas en los resultados experimentales

Aspectos a cubrir:

### 1. Tiempo de Ejecución
TODO: Analizar tabla de tiempos consolidada
- Comparar speedups TIME vs MEMORY
- Evaluar consistencia vs Q1
- Identificar bottlenecks específicos de extracción de emojis

### 2. Uso de Memoria (Delta RSS)
TODO: Analizar tabla de memoria consolidada
- Comparar deltas TIME vs MEMORY
- Evaluar overhead del Counter
- Comparar con Q1 para identificar diferencias

### 3. Escalabilidad Esperada
TODO: Proyectar comportamiento con datasets más grandes
- Evaluar cómo escala emoji.emoji_list() con más tweets
- Predecir impacto en memoria y tiempo
- Comparar con Q1

### 4. Trade-offs Arquitecturales
TODO: Evaluar cuándo usar cada enfoque
- Polars TIME vs Polars MEMORY
- Pandas TIME vs Pandas MEMORY
- Comparación cruzada

### 5. Recomendación Final
TODO: Recomendar enfoque óptimo para Q2
- Considerar tiempo, memoria y escalabilidad
- Comparar con recomendación de Q1
- Justificar decisión con datos experimentales

### 6. Limitaciones y Trabajos Futuros
TODO: Identificar limitaciones del análisis actual
- Limitaciones de emoji.emoji_list() (row-by-row processing)
- Posibles optimizaciones (UDFs, paralelización)
- Alternativas a la librería emoji
- Impacto de emojis compuestos

### Conclusión Final

TODO: Resumir hallazgos principales y recomendación definitiva para Q2 en producción.